## Instructions

1. Log in to Google
2. Input the .csv filename that will be uploaded
3. From the "Runtime" menu, click "Run all"
4. Upload the .csv file

The code will plot the data and download a .zip file containing the plots.

In [ ]:
#@title
from google.colab import files
files.upload()

In [7]:
filename = "Sample" #@param {type: "string"}

In [ ]:
#@title
import os
import math
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point



def set_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])



# Get the raw data
directory = "/content"
filepath = os.path.join(directory, filename + ".csv")

# Check that the file exists
if not os.path.exists(filepath):
    raise Exception(str(filepath) + ' does not exist')
    
# Read the data
raw_data = pd.read_csv(filepath, header = None)

# Identify each survey data chunk separated by a blank row
idx_split = list(raw_data.index[raw_data.isnull().all(1)])

# Check that there are at least two separating lines
if len(idx_split) < 2:
    raise Exception('Insufficient blank lines found separating survey sections')

# Split raw data
survey_data = []
for df in np.split(raw_data, idx_split):
    df = df.dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all')
    df = df.rename(columns = df.iloc[0]).drop(df.index[0])
    survey_data.append(df)



# Pop specific items
project_info = survey_data.pop(0)
fixes = survey_data.pop(0)

# Check project header
if list(project_info.columns) != ['Name', 'MagDevEast']:
    raise Exception('Project header is missing or malformed')

if len(project_info) == 0:
    raise Exception('No project information is supplied')

try:
    project_info['MagDevEast'] = project_info['MagDevEast'].astype(float)
except:
    raise Exception("'MagDevEast' is non-numeric")

# Check fix header
if list(fixes.columns) != ['Station', 'FixX', 'FixY', 'FixZ']:
    raise Exception('Fix header is missing or malformed')

if len(fixes) == 0:
    raise Exception('No fixes are supplied, provide at least one of coordinates 0, 0, 0')

for i in ['FixX', 'FixY', 'FixZ']:
    try:
        fixes[i] = fixes[i].astype(float)
    except:
        raise Exception("'" + i + "' contains non-numeric values")

try:
    fixes['Station'] = fixes['Station'].astype(str)
except:
    raise Exception("'Station' contains non-string values")



# Stations to process
stations_waiting = pd.concat(survey_data)
    
# Check column names
if list(stations_waiting.columns)[:5] != ['FromStation', 'ToStation', 'Dist', 'Azim', 'Incl']:
    raise Exception('Survey header is missing')    

# Check for blanks
for i in ['FromStation', 'ToStation', 'Dist', 'Azim', 'Incl']:
    if stations_waiting[i].isna().sum() > 0:
        raise Exception("'" + i + "' contains blank values")

# Set data type
for i in ['FromStation', 'ToStation']:
    try:
        stations_waiting[i] = stations_waiting[i].astype(str)
    except:
        raise Exception("'" + i + "' contains non-string values")

for i in ['Dist', 'Azim', 'Incl']:
    try:
        stations_waiting[i] = stations_waiting[i].astype(float)
    except:
        raise Exception("'" + i + "' contains non-numeric values")

# Check for valid ranges
if not all(0 < i for i in stations_waiting['Dist']):
    raise Exception('Invalid distance values')
    
if not all(0 <= i <= 360 for i in stations_waiting['Azim']):
    raise Exception('Invalid azimuth values')
    
if not all(-90 <= i <= 90 for i in stations_waiting['Incl']):
    raise Exception('Invalid inclination values')

# Correct azimuths to grid north
stations_waiting['Azim'] = stations_waiting['Azim'].apply(lambda x: (x + project_info['MagDevEast'].iloc[0])%360)



# Calculated stations
stations = {}

# Add all fixes as calculated stations
for _, fix in fixes.iterrows():
    stations[fix['Station']] = Point(fix['FixX'], fix['FixY'], fix['FixZ'])
    
# Calculated legs
legs = []

# Temp lengths
num_start = len(stations_waiting)
num_end = len(stations_waiting) - 1

# Loop continuously
while True:
    # End if all stations are processed or if no stations to process appear in the calculated stations
    if len(stations_waiting) == 0 or len(set(stations_waiting['FromStation']).union(stations_waiting['ToStation']).intersection(set(stations.keys()))) == 0:
        break
    
    num_start = len(stations_waiting)
    
    # Loop through all stations to process
    for idx, row in stations_waiting.iterrows():
        # If FromStation is already calculated
        if row['FromStation'] in stations.keys():
            # Get FromStation point
            from_station_point = stations.get(row['FromStation'])
                        
            # If ToStation is already calculated
            if row['ToStation'] in stations.keys():
                # Get ToStation point, skipping calculation
                to_station_point = stations.get(row['ToStation'])
            # If ToStation is new
            else:
                # Calculate ToStation point using spherical to cartesian coordinate conversion
                azimuth_rads = math.radians(row['Azim'])
                inclination_rads = math.radians(row['Incl'])

                to_station_point = Point(from_station_point.x + (row['Dist']*math.sin(azimuth_rads)*math.cos(inclination_rads)),
                                         from_station_point.y + (row['Dist']*math.cos(azimuth_rads)*math.cos(inclination_rads)),
                                         from_station_point.z + (row['Dist']*math.sin(inclination_rads)))
                
                # Log ToStation point as calculated if not a splay
                if row['ToStation'] != "#": stations[row['ToStation']] = to_station_point
            
            # Log calculated survey leg
            legs.append((row['FromStation'] + ' to ' + row['ToStation'],
                         LineString([from_station_point, to_station_point])))
            
            # Drop from processing
            stations_waiting.drop(idx, inplace = True)
            
    num_end = len(stations_waiting)
    
    # If there have been no stations linked
    if num_start == num_end:
        # Reverse stations and repeat
        stations_waiting['ToStationTemp'] = stations_waiting['ToStation']
        stations_waiting['FromStationTemp'] = stations_waiting['FromStation']
        stations_waiting['FromStation'] = stations_waiting.pop('ToStationTemp')
        stations_waiting['ToStation'] = stations_waiting.pop('FromStationTemp')
        stations_waiting['Azim'] = stations_waiting['Azim'].apply(lambda x: x + 180 if x < 180 else x - 180)
        stations_waiting['Incl'] = stations_waiting['Incl'].apply(lambda x: x*-1 if x != 0 else x)

# Print any unlinked stations
if len(stations_waiting) > 0:
    print("Unlinked stations:\n" + "\n".join(set(stations_waiting['FromStation']).union(stations_waiting['ToStation'])))



# Create an output folder
from pathlib import Path
output_directory = os.path.join(directory, 'plots')
Path(output_directory).mkdir(parents = True, exist_ok = True)

# Plot
import matplotlib.pyplot as plt
for figure_title, view in zip(["Top Down (North up)", "Side (Looking North)", "Side (Looking West)", '3D'],
                        [(90, 270), (0, 270), (0, 0), (20, 225)]):

    fig = plt.figure(figsize = (20, 20))
    ax = fig.add_subplot(111, projection = '3d', proj_type = 'ortho')
    ax.set_title(project_info['Name'].iloc[0] + "\n" + figure_title, y = 1, fontsize = 20)

    # Plot legs
    for leg, line in legs:
        if "#" in leg:
            linewidth = 0.3
            colour = 'blue'
            alpha = 0.5
        else:
            linewidth = 1
            colour = 'black'
            alpha = 1
        ax.plot([i[0] for i in line.coords], [i[1] for i in line.coords], [i[2] for i in line.coords],
                linewidth = linewidth, color = colour, alpha = alpha) 
    
    # Plot and label stations
    for station, point in stations.items():
        ax.scatter(point.x, point.y, point.z,
                   s = 2, color = 'black', depthshade = False)
        ax.text(point.x + 0.5, point.y + 0.5, point.z + 0.5,
                str(station), fontsize = 3, color = 'red')
    
    ax.view_init(elev = view[0], azim = view[1])
    set_axes_equal(ax)
    plt.savefig(os.path.join(output_directory, project_info['Name'].iloc[0] + "_" + figure_title + ".jpeg"),
                dpi = 600, bbox_inches = 'tight')
    plt.show()

In [9]:
#@title
!zip -r /content/plots.zip /content/plots

In [10]:
#@title
from google.colab import files
files.download("/content/plots.zip")